In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Esse arquivo é ignorado pelo Git. O script somente deve ser usado para testar gráficos antes de inserir no arquivo indicadores.py**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import requests\n",
    "import sidrapy\n",
    "import numpy as np\n",
    "import scripts_saneamento\n",
    "import datetime as dt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 130,
   "metadata": {},
   "outputs": [],
   "source": [
    "ipca = scripts_saneamento.importa_ipca('ipca_202210SerieHist.xls')\n",
    "\n",
    "ipca['Mês'] = pd.to_datetime(ipca['Data'], format='%m/%Y')\n",
    "\n",
    "ipca.set_index('Mês', inplace=True)\n",
    "\n",
    "ipca['index'] = ipca.index.year\n",
    "\n",
    "ipca['IPCA - Variação mensal'] = ipca['Índice'].pct_change(periods=1)*100\n",
    "\n",
    "ipca['IPCA - Variação acumulada em 12 meses'] = ipca['Índice'].pct_change(periods=12)*100\n",
    "\n",
    "ipca[\"IPCA - Variação acumulada no ano\"] = ipca.groupby([ipca.index.year])[\"IPCA - Variação mensal\"].transform(sum)\n",
    "\n",
    "for mes in ipca.index.month:\n",
    "    ipca['IPCA - Variação acumulada no ano'] = ipca['IPCA - Variação mensal'].rolling(mes).sum()\n",
    "\n",
    "ipca['ano_base'] = ipca['Índice'].groupby(ipca.index.year).transform('last')\n",
    "\n",
    "ipca['ano_base'] = ipca['ano_base'].shift(12)\n",
    "\n",
    "ipca['IPCA - Variação acumulada no ano'] = (ipca['Índice']/ipca['ano_base'] - 1) * 100\n",
    "\n",
    "\n",
    "ipca = ipca[(ipca.index.year > 1994) & (ipca.index.year < 2022)].drop(['index', 'Índice', 'ano_base', 'Data'], axis=1)\n",
    "\n",
    "ipca['IPCA - Variação acumulada no ano'] = pd.to_numeric(ipca['IPCA - Variação acumulada no ano'])\n",
    "\n",
    "ipca['IPCA - Variação acumulada no ano'] = ipca['IPCA - Variação acumulada no ano'].round(2)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 131,
   "metadata": {},
   "outputs": [],
   "source": [
    "ipca.reset_index(inplace=True)\n",
    "ipca.to_feather('ipca_brasil_94_21.feather')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 85,
   "metadata": {},
   "outputs": [],
   "source": [
    "data_atual = dt.datetime.now() # captura data para determinar período na consulta da API\n",
    "ipca_brasil = requests.get(f'https://apisidra.ibge.gov.br/values/t/7060/n1/all/v/63,69,2265/p/202201-{data_atual.year}{data_atual.month-1}/c315/7169/d/v63%202')\n",
    "ipca_brasil = ipca_brasil.json()\n",
    "ipca_brasil = pd.DataFrame(ipca_brasil)\n",
    "\n",
    "## Tratamento dos dados do IPCA (Variação Mensal, Acumulado no Ano e Acumulado em 12 Meses)\n",
    "\n",
    "ipca_brasil.columns = ipca_brasil.iloc[0] \n",
    "ipca_brasil = ipca_brasil[1:]\n",
    "ipca_brasil['Valor'] = ipca_brasil['Valor'].astype(float)\n",
    "ipca_brasil['Mês (Código)'] = pd.to_datetime(ipca_brasil['Mês (Código)'], format='%Y%m')\n",
    "\n",
    "ipca_brasil = pd.crosstab(ipca_brasil['Mês (Código)'], ipca_brasil['Variável'], ipca_brasil['Valor'], aggfunc='sum')\n",
    "\n",
    "coluna_ordem = ['IPCA - Variação mensal', 'IPCA - Variação acumulada em 12 meses', 'IPCA - Variação acumulada no ano']\n",
    "\n",
    "ipca_brasil = ipca_brasil.reindex(coluna_ordem, axis=1)\n",
    "\n",
    "ipca_brasil_94_21 = pd.read_feather('ipca_brasil_94_21.feather')\n",
    "\n",
    "ipca_brasil_94_21.set_index('Mês', inplace=True)\n",
    "\n",
    "ipca_brasil = pd.concat([ipca_brasil_94_21, ipca_brasil], axis=0)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 144,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "2022"
      ]
     },
     "execution_count": 144,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ipca_brasil.index.max().year"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 145,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>IPCA - Variação mensal</th>\n",
       "      <th>IPCA - Variação acumulada em 12 meses</th>\n",
       "      <th>IPCA - Variação acumulada no ano</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>2022-01-01</th>\n",
       "      <td>0.54</td>\n",
       "      <td>10.38</td>\n",
       "      <td>0.54</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-02-01</th>\n",
       "      <td>1.01</td>\n",
       "      <td>10.54</td>\n",
       "      <td>1.56</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-03-01</th>\n",
       "      <td>1.62</td>\n",
       "      <td>11.30</td>\n",
       "      <td>3.20</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-04-01</th>\n",
       "      <td>1.06</td>\n",
       "      <td>12.13</td>\n",
       "      <td>4.29</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-05-01</th>\n",
       "      <td>0.47</td>\n",
       "      <td>11.73</td>\n",
       "      <td>4.78</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-06-01</th>\n",
       "      <td>0.67</td>\n",
       "      <td>11.89</td>\n",
       "      <td>5.49</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-07-01</th>\n",
       "      <td>-0.68</td>\n",
       "      <td>10.07</td>\n",
       "      <td>4.77</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-08-01</th>\n",
       "      <td>-0.36</td>\n",
       "      <td>8.73</td>\n",
       "      <td>4.39</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-09-01</th>\n",
       "      <td>-0.29</td>\n",
       "      <td>7.17</td>\n",
       "      <td>4.09</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2022-10-01</th>\n",
       "      <td>0.59</td>\n",
       "      <td>6.47</td>\n",
       "      <td>4.70</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "            IPCA - Variação mensal  IPCA - Variação acumulada em 12 meses  IPCA - Variação acumulada no ano\n",
       "2022-01-01                    0.54                                  10.38                              0.54\n",
       "2022-02-01                    1.01                                  10.54                              1.56\n",
       "2022-03-01                    1.62                                  11.30                              3.20\n",
       "2022-04-01                    1.06                                  12.13                              4.29\n",
       "2022-05-01                    0.47                                  11.73                              4.78\n",
       "2022-06-01                    0.67                                  11.89                              5.49\n",
       "2022-07-01                   -0.68                                  10.07                              4.77\n",
       "2022-08-01                   -0.36                                   8.73                              4.39\n",
       "2022-09-01                   -0.29                                   7.17                              4.09\n",
       "2022-10-01                    0.59                                   6.47                              4.70"
      ]
     },
     "execution_count": 145,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ipca_brasil[(ipca_brasil.index >= f'{ipca_brasil.index.max().year}-01-01')]"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3.9.7 ('base')",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  },
  "orig_nbformat": 4,
  "vscode": {
   "interpreter": {
    "hash": "08a400adeb6830322ec40344573c657b15a9eb2eef6e1c7ab2815bda50c98d3e"
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}